# The Application of Graph Convolutional Network (GCN)

### It is a method for semi-supervised learning on graph-structured data. It is based on an efficient variant of convolutional neural networks (CNNs) which operate directly on graphs.

In [ ]:
import math
import time
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as sp
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden = 16                                     # hidden dimension
dropout = 0.5
lr = 0.01 
weight_decay = 5e-4
fastmode = 'store_true'

In [9]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def normalize(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def  load_data(path="/Users/wushuangyan/Desktop/STAT classes/Data Science/HW4/", dataset="cora"):
    print('Loading {} dataset...'.format(dataset))
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), # read node labels
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)  # read node features
    labels = encode_onehot(idx_features_labels[:, -1])                        # one-hot encoding for labels
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)                
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),       # read edge information
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    features = normalize(features)                                            # feature normalization
    adj = normalize(adj + sp.eye(adj.shape[0]))                               # edge normalization

    idx_train = range(140)                                                    # training set
    idx_val = range(200, 500)                                                 # validation set
    idx_test = range(500, 1500)                                               # testing set

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)                               # get adjacency matrix

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
                                                           
    return adj, features, labels, idx_train, idx_val, idx_test          

adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [10]:
class GraphConvolution(Module):                            
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):             # AHW 
        support = torch.mm(input, self.weight) # (2708, 16) = (2708, 1433) X (1433, 16)
        output = torch.spmm(adj, support)      # (2708, 16) = (2708, 2708) X (2708, 16)
        return output + self.bias

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN(nn.Module):                                             # a two-layer GCN
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = torch.nn.functional.relu(self.gc1(x, adj))
        x = torch.nn.functional.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return torch.nn.functional.log_softmax(x, dim=1)          # softmax

model = GCN(nfeat=features.shape[1], nhid=hidden,
            nclass=labels.max().item() + 1,dropout=dropout)

In [11]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)                         
    loss_train = torch.nn.functional.nll_loss(output[idx_train], labels[idx_train])  # loss function
    acc_train = accuracy(output[idx_train], labels[idx_train])                       # training accuracy
    loss_train.backward()                                                            # backpropagation
    optimizer.step()                                                                 # update parameters

    if not fastmode:
        model.eval()
        output = model(features, adj)

    loss_val = torch.nn.functional.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [12]:
def test():
    model.eval()
    output = model(features, adj)                     # features:(2708, 1433)   adj:(2708, 2708)
    loss_test = torch.nn.functional.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    
optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)

epochs = 500
for epoch in range(epochs):
    train(epoch)
test()

Epoch: 0001 loss_train: 2.0425 acc_train: 0.1429 loss_val: 2.0737 acc_val: 0.1233 time: 0.0334s
Epoch: 0002 loss_train: 2.0202 acc_train: 0.1500 loss_val: 2.0636 acc_val: 0.1200 time: 0.0133s
Epoch: 0003 loss_train: 2.0012 acc_train: 0.1500 loss_val: 2.0414 acc_val: 0.1267 time: 0.0132s
Epoch: 0004 loss_train: 1.9895 acc_train: 0.1571 loss_val: 2.0285 acc_val: 0.1300 time: 0.0150s
Epoch: 0005 loss_train: 1.9719 acc_train: 0.2357 loss_val: 2.0072 acc_val: 0.1767 time: 0.0161s
Epoch: 0006 loss_train: 1.9578 acc_train: 0.2500 loss_val: 1.9950 acc_val: 0.1633 time: 0.0144s
Epoch: 0007 loss_train: 1.9560 acc_train: 0.2071 loss_val: 1.9835 acc_val: 0.1633 time: 0.0154s
Epoch: 0008 loss_train: 1.9330 acc_train: 0.2000 loss_val: 1.9697 acc_val: 0.1567 time: 0.0160s
Epoch: 0009 loss_train: 1.9115 acc_train: 0.2000 loss_val: 1.9526 acc_val: 0.1567 time: 0.0159s
Epoch: 0010 loss_train: 1.9109 acc_train: 0.2000 loss_val: 1.9406 acc_val: 0.1567 time: 0.0182s
Epoch: 0011 loss_train: 1.9011 acc_train